In [1]:

import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

import numpy as np
import torch
#import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# from sklearn.model_selection import train_test_split
import tqdm
import matplotlib.pyplot as plt
import logging

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.chdir('../')

from pose_gru import PoseGRU_inputFC2
from benji_prox_dataloader import *

numpy: 1.21.5


In [13]:
root_dir = "D:/prox_data/joint_locations/"
smplx_model_path='../models_smplx_v1_1/models/'
in_frames = 10
pred_frames = 5
batch_size = 15
pd = proxDatasetJoints(root_dir)


dataloader = DataLoader(pd, batch_size=batch_size,
                        shuffle=True, num_workers=0, collate_fn=my_collate)

In [15]:
pd.__getitem__(100)

KeyError: 'in_joint_locations'

In [5]:
from unittest.mock import Mock
self = Mock()

output_type='raw_pkls'

self.output_type = output_type
self.root_dir = Path(root_dir)

scenes = glob.glob(str(self.root_dir / '*'))
scenes = [Path(scene) for scene in scenes]
scenes_dir = {scene.name: list(map(Path, glob.glob(str(scene / 'results/*')))) for scene in scenes}
scenes_dir2 = {stem: [(str(path / '000.pkl'), path.name)  for path in paths] for stem, paths in scenes_dir.items()}
outputs = {}
for stem, paths in scenes_dir2.items():
    if not stem in outputs:
        outputs[stem] = {}
    for path, name in paths:
        scene, frame, tstamp = re.match(r's(\d*)_frame_(\d*)__(.*)', name).groups()
        scene, frame = int(scene), int(frame)
        tstamp = dt.datetime.strptime(tstamp, '%H.%M.%S.%f').time()
        if scene not in outputs[stem]:
            outputs[stem][scene] = []
        outputs[stem][scene].append({'fn': path, 'frame': frame, 'tstamp': tstamp})

In [8]:
outputs

{'BasementSittingBooth_00142_01': {1: [{'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00010__00.00.00.305\\000.pkl',
    'frame': 10,
    'tstamp': datetime.time(0, 0, 0, 305000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00025__00.00.00.799\\000.pkl',
    'frame': 25,
    'tstamp': datetime.time(0, 0, 0, 799000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00040__00.00.01.299\\000.pkl',
    'frame': 40,
    'tstamp': datetime.time(0, 0, 1, 299000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00055__00.00.01.797\\000.pkl',
    'frame': 55,
    'tstamp': datetime.time(0, 0, 1, 797000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00070__00.00.02.299\\000.pkl',
    'frame': 70,
    'tstamp': datetime.time(0, 0, 2, 299000)}]},
 'N3Office_03301_

In [9]:
sequences = []
for stem in outputs:
    for scene, frame_dicts in outputs[stem].items():
        sequences.append([stem, sorted(frame_dicts, key=lambda x: x['frame'])])

In [10]:
sequences

[['BasementSittingBooth_00142_01',
  [{'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00010__00.00.00.305\\000.pkl',
    'frame': 10,
    'tstamp': datetime.time(0, 0, 0, 305000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00025__00.00.00.799\\000.pkl',
    'frame': 25,
    'tstamp': datetime.time(0, 0, 0, 799000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00040__00.00.01.299\\000.pkl',
    'frame': 40,
    'tstamp': datetime.time(0, 0, 1, 299000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00055__00.00.01.797\\000.pkl',
    'frame': 55,
    'tstamp': datetime.time(0, 0, 1, 797000)},
   {'fn': 'D:\\prox_data\\joint_locations\\BasementSittingBooth_00142_01\\results\\s001_frame_00070__00.00.02.299\\000.pkl',
    'frame': 70,
    'tstamp': datetime.time(0, 0, 2, 299000)}]],
 ['N3Office_03301_

In [11]:
self.sequences = sequences
self.in_frames = in_frames
self.pred_frames = pred_frames
self.tot_frames = in_frames + pred_frames
seq_lens = [len(fns_dict) for stem, fns_dict in sequences]
self.bounds = np.array([seq_len // self.tot_frames for seq_len in seq_lens])  # e.g. 
assert(np.all(self.bounds >= 1)), "sequence has insufficient frames for one training input"  # sanity check

AssertionError: sequence has insufficient frames for one training input

In [12]:
seq_lens

[5, 136]

In [5]:
seq_lens = [len(fns_dict) for stem, fns_dict in pd.sequences]

In [8]:
seq_lens = np.array(seq_lens)

In [10]:
sl = np.cumsum(seq_lens)

In [11]:
sl

array([ 1581,  3763,  6057,  8231, 10418, 11876, 13790, 15627, 17379,
       18669, 19910, 21675, 23164, 24949, 26466, 28677, 32132, 33610,
       36681, 38708, 40192, 41288, 42531, 43619, 44972, 48010, 49456,
       51722, 53826, 54793, 55507, 56334, 56986, 58086, 58550, 59220,
       60204, 62356, 63693, 65947, 68546, 71606, 73643, 75687, 76683,
       78008, 79319, 81234, 82290, 83162, 84066, 84967, 86958, 88489],
      dtype=int32)

In [12]:
np.sum(seq_lens)

88489

In [14]:
pd.sequences[1][1][5]

{'fn': 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00145_01\\results\\s001_frame_00006__00.00.00.153\\000.pkl',
 'frame': 6,
 'tstamp': datetime.time(0, 0, 0, 153000)}

In [18]:
fn = r'D:\prox_data\joint_locations\N3Office_03301_01\results\s001_frame_00010__00.00.00.303\000.pkl'
with open(fn, 'rb') as file:
    stuff = pickle.load(file)

In [19]:
stuff

{'in_fns': ['/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00001__00.00.00.027/000.pkl',
  '/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00002__00.00.00.044/000.pkl',
  '/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00003__00.00.00.068/000.pkl',
  '/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00004__00.00.00.105/000.pkl',
  '/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00005__00.00.00.137/000.pkl',
  '/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00006__00.00.00.177/000.pkl',
  '/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00007__00.00.00.206/000.pkl',
  '/cluster/scratch/bdayan/prox_data/PROXD_attempt2/PROXD/N3Office_03301_01/results/s001_frame_00008__00.00.00.236/

In [20]:
stuff.keys()

dict_keys(['in_fns', 'pred_fns', 'in_joint_locations', 'pred_joint_locations'])